# Установим необходимые пакеты

Для работы нам понадобится лемматайзер, который может обрабатывать корпуса текстов на русском. Запустить через колаб Mystem от pymystem3 у меня не получилось, поэтому воспользуемся pymorphy2

In [1]:
!pip install pymorphy2

     |████████████████████████████████| 55 kB 2.1 MB/s 
     |████████████████████████████████| 8.2 MB 8.5 MB/s 


In [2]:
import nltk
from nltk.corpus import stopwords
import string

import pymorphy2
from pymorphy2 import MorphAnalyzer

import pandas as pd
import numpy as np

In [3]:
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
sw_rus = set(stopwords.words('russian'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


Создадим тренировочный небольшой набор данных, на котором можно было бы обучить наш классификатор

In [19]:
letter = 'купи средство от геморроя, и получи 1 виагру за полцены'
letter2 = 'Иван, срочно поменяй пароль - тебя взломали!'
letter3 = 'Купи Иван средство от геморроя да колись ты в рот'

In [20]:
df_text = pd.DataFrame(data=[[letter, 'spam'], [letter2, 'ham'], [letter3, 'spam']], columns=['letter', 'label'])
df_text

,letter,label
0,"купи средство от геморроя, и получи 1 виагру з...",spam
1,"Иван, срочно поменяй пароль - тебя взломали!",ham
2,Купи Иван средство от геморроя да колись ты в рот,spam


Формулы, которые я находил, немного отличаются друг от друга (в основном из-за знаменателя формулы Байеса и сглаживания. Я остановился на этом варианте:

https://habr.com/ru/post/415963/

In [13]:
class NaiveBayesClassifier():
  def __init__(self, alpha):
    self.alpha = alpha # параметр сглаживания
    self.count_spam = 0 # количество spam писем
    self.count_ham = 0 # количество ham писем
    self.dict_spam = {} # словарь ("слово": сколько оно встречалось в spam)
    self.dict_ham = {} # словарь ("слово":сколько оно встречалось в ham)
    self.morph = MorphAnalyzer(lang="ru")


  def clean_text(self, text):
    text = text.lower()
    text = [word.strip(string.punctuation) for word in text.split(" ")]
    text = [word for word in text if not any(c.isdigit() for c in word)]
    text = [x for x in text if x not in sw_rus]
    text = [t for t in text if len(t) > 0]
    tokens = [self.morph.parse(word)[0].normal_form for word in text]
    return tokens


  def dict_completion(self, letter, label):
    tokens = self.clean_text(letter)
    if label == 'ham':
      d = self.dict_ham
    elif label == 'spam':
      d = self.dict_spam

    for word in tokens:
      if word in d.keys():
        d[word] += 1
      else:
        d[word] = 1
  
  def fit(self, letters, labels):
    self.count_spam += len(labels[labels == 'spam'])
    self.count_ham += len(labels[labels == 'ham'])

    for letter, label in zip(letters, labels):
      self.dict_completion(letter, label)

  def predict_proba_spam(self, letters): # будем предсказывать вероятность того, что письмо относится к спаму - P (spam | letter) = P(spam) * П ( P(word | spam) ) / П (P (word)). Знаменатель не рассматриваем.
    list_of_labels = []
    for letter in letters:
      prod_num = (self.count_spam / (self.count_ham + self.count_spam)) # P(spam) - числитель
      tokens = self.clean_text(letter)
      prod_den = 1
      for word in tokens:
        if word in self.dict_spam.keys():
          prod_num *= ((self.dict_spam[word] + self.alpha) / (self.alpha * (len(self.dict_spam) + len(self.dict_ham)) + len(self.dict_spam))) # П ( P(word | spam) )
        else:
          prod_num *= (self.alpha / (self.alpha * (len(self.dict_spam) + len(self.dict_ham)) + len(self.dict_spam)))
      list_of_labels.append(prod_num / prod_den)
    return np.array(list_of_labels)


  def predict_proba_ham(self, letters):
    list_of_labels = []
    for letter in letters:
      prod_num = (self.count_ham / (self.count_ham + self.count_spam))
      tokens = self.clean_text(letter)
      prod_den = 1
      for word in tokens:
        if word in self.dict_ham.keys():
          prod_num *= ((self.dict_ham[word] + self.alpha) / (self.alpha * (len(self.dict_spam) + len(self.dict_ham)) + len(self.dict_ham))) # П ( P(word | spam) )
        else:
          prod_num *= (self.alpha / (self.alpha * (len(self.dict_spam) + len(self.dict_ham)) + len(self.dict_ham)))
      list_of_labels.append(prod_num / prod_den)
    return np.array(list_of_labels)

  def predict_proba(self, letters):
    list_of_labelz = []
    for letter in letters:
      pred_spam = (self.predict_proba_spam(np.array([letter]))[0] / (self.predict_proba_spam(np.array([letter]))[0] + self.predict_proba_ham(np.array([letter]))[0]))
      list_of_labelz.append(pred_spam)
    return np.array(list_of_labelz)

Теперь создадим тренировочную выборку

In [21]:
test = pd.DataFrame(data=np.array(['геморрой получить срочно', 'геморрой получить срочно']))
test

,0
0,геморрой получить срочно
1,геморрой получить срочно


In [22]:
NaiveBayes = NaiveBayesClassifier(alpha=1)
NaiveBayes.fit(df_text['letter'], df_text['label'])

Предсказание вероятности спама для тестовой выборки

In [24]:
NaiveBayes.predict_proba_spam(test[0])

array([0.00032876, 0.00032876])

Предсказание вероятности класса "не спам" для тестовой выборки

In [25]:
NaiveBayes.predict_proba_ham(test[0])

array([9.71958983e-05, 9.71958983e-05])

Числа довольно маленькие, да и не отражают значения вероятности в том смысле, в котором мы привыкли эти вероятности видеть (находятся от 0 до 1, ближе к 1 означает бОльшую вероятость и т.д.)

Введем новую величину - $P_{new} = \frac{P(spam|letter)}{P(spam|letter) + P(ham|letter)}$

Она позволит представить сумму вероятностей принадлежности к двум классам (спам / не спам) как единицу, а сама величина покажет, какую часть от этой суммы занимает вероятность принадлежности спаму

In [26]:
NaiveBayes.predict_proba(test[0])

array([0.77181598, 0.77181598])